In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import os
from collections import defaultdict
from collections import Counter

import matplotlib.pyplot as plt

In [2]:
import psutil
psutil.cpu_count()
p = psutil.Process()
p.cpu_affinity([0,1,2,3,4,5,6,7,8])

In [3]:
os.chdir(r'H:\all\RL_Shrinkage_2024')
from helpers import helper_functions as hf

In [4]:
import glob

path = r'H:\all\RL_Shrinkage_2024\ONE_YR\NonLinear_Shrinkage\transformed_qis_eigenvalues'
extension = 'csv'
os.chdir(path)
filenames = glob.glob('*.{}'.format(extension))
print(filenames)

['cov2para_evs_df.csv', 'cov2para_fixed04_evs_df.csv', 'cov2para_plus02_evs_df.csv', 'qis_evs_df.csv', 'qis_evs_df_p100.csv', 'qis_evs_df_p225.csv', 'qis_evs_df_p30.csv', 'qis_evs_df_p50.csv', 'qis_evs_df_p500.csv', 'qis_evs_exp_05.csv', 'qis_evs_exp_2.csv', 'sample_evs_df.csv', 'sample_evs_df_p100.csv', 'sample_evs_df_p225.csv', 'sample_evs_df_p30.csv', 'sample_evs_df_p50.csv', 'sample_evs_df_p500.csv']


In [5]:
def get_ev_intensities(evs, sample_evs):
    intensities = evs.copy()
    if sample_evs.shape[1] == 500:
        intensities.iloc[:, -251:] = evs.iloc[:, -251:].values / sample_evs.iloc[:, -251:].values
    else:
        intensities = evs.values / sample_evs.values
    return intensities

def plot_last_x_intensities_across_time(intensities_dfs, modelnames, num_evs, return_df=False):
    if len(modelnames) > 1:
        dfs_list = [intensities_dfs[M].iloc[:, -num_evs:].mean(axis=1) for M in modelnames]
        df = pd.concat(dfs_list, axis=1)
        df.columns = modelnames
    else:
        df = intensities_dfs[modelnames[0]].iloc[:, -num_evs:].mean(axis=1)
        df.name = modelnames[0]
    fig = px.line(
        df,
        title = f"Plot of Average EV(Model) / EV(Sample) Across Time",
         labels={'value':f'Mean of EV(Model) / EV(Sample)'},
        height=700
    )
    fig.show()

def plot_last_x_intensities_across_assets(intensities_dfs, modelnames, num_evs, return_df=False):
    if len(modelnames) > 1:
        dfs_list = [intensities_dfs[M].iloc[:, -num_evs:].mean() for M in modelnames]
        df = pd.concat(dfs_list, axis=1)
        df.columns = modelnames
    else:
        df = intensities_dfs[modelnames[0]].iloc[:, -num_evs:].mean()
        df.name = modelnames[0]
        
    fig = px.line(
        df,
        title = f"Plot of Average EV(Model) / EV(Sample) Across Assets",
         labels={'value':f'Mean of EV(Model) / EV(Sample)', 'index': "Eigenvalue"},
        height=700
    )
    fig.show()


def plot_last_x_eigenvalues_across_time(eigenvalues_dfs, modelnames, ev_range, return_df=False):
    if len(modelnames) > 1:
        dfs_list = [eigenvalues_dfs[M].iloc[:, ev_range[0]:ev_range[1]].mean(axis=1) for M in modelnames]
        df = pd.concat(dfs_list, axis=1)
        df.columns = modelnames
    else:
        df = eigenvalues_dfs[modelnames[0]].iloc[:, ev_range[0]:ev_range[1]].mean(axis=1)
        df.name = modelnames[0]
    fig = px.line(
        df,
        title = f"Plot of Average EV(Model) Across Time",
         labels={'value':f'Mean of EV(Model) '},
        height=700
    )
    fig.show()

def plot_last_x_eigenvalues_across_assets(eigenvalues_dfs, modelnames, ev_range, return_df=False):
    if len(modelnames) > 1:
        dfs_list = [eigenvalues_dfs[M].iloc[:, ev_range[0]:ev_range[1]].mean() for M in modelnames]
        df = pd.concat(dfs_list, axis=1)
        df.columns = modelnames
    else:
        df = eigenvalues_dfs[modelnames[0]].iloc[:, ev_range[0]:ev_range[1]].mean()
        df.name = modelnames[0]
        
    fig = px.line(
        df,
        title = f"Plot of Average EV(Model) Across Assets",
         labels={'value':f'Mean of EV(Model)', 'index': "Eigenvalue"},
        height=700
    )
    fig.show()


def get_rawres_LONG_only(eigenvalue_dict, modelnames: list):
    tmp_res = defaultdict(list)
    tmp_rawres = defaultdict(list)
    for idx in range(0, 1):
        try:
            past_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + add_idx - 21 * 12 * 1: idx + add_idx, :]
            past_ret_mat = past_ret_mat.sub(past_ret_mat.mean())
            past_ret_mat = past_ret_mat.fillna(0)
            fut_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + add_idx: idx + add_idx + 21, :]
        except:
            print("Some Error..")
            
        N, p = past_ret_mat.shape
        sample = pd.DataFrame(np.matmul(past_ret_mat.T.to_numpy(), past_ret_mat.to_numpy())) / (N - 1)
        lambda1, u = np.linalg.eigh(sample)
        lambda1 = lambda1.real.clip(min=0)
        dfu = pd.DataFrame(u,columns=lambda1)
        dfu.sort_index(axis=1,inplace = True)
        temp1 = dfu.to_numpy()
        temp3 = dfu.T.to_numpy().conjugate()

        for cur_modelname in modelnames:
            qis = eigenvalue_dict[cur_modelname].iloc[idx, :]
            temp2 = np.diag(qis)
            sigmahat = pd.DataFrame(np.matmul(np.matmul(temp1, temp2), temp3))
            try:
                weights = hf.calc_global_min_variance_pf_long_only(sigmahat)
            except:
                print("Some Other Error..")
            # store results
            tmp_res[cur_modelname].append(np.std(fut_ret_mat @ weights, ddof=1) * np.sqrt(252) * 100)
            if idx % 21 == 0:
                tmp_rawres[cur_modelname] += list(fut_ret_mat @ weights)

    return tmp_rawres, tmp_res
    

In [6]:
# LOAD EIGENVALUES
PF_SIZE = 500

model_names = ["qis",  "sample"]
evs_dfs = {}
qis_evs_path = r"H:\all\RL_Shrinkage_2024\ONE_YR\NonLinear_Shrinkage\transformed_qis_eigenvalues"
evs_dfs["qis"] = pd.read_csv(qis_evs_path + f'\\qis_evs_df_p{PF_SIZE}.csv', index_col=0)
evs_dfs["sample"] = pd.read_csv(qis_evs_path + f'\\sample_evs_df_p{PF_SIZE}.csv', index_col=0)

base_folder_path = r'H:\\all\\RL_Shrinkage_2024'
permnos = pd.read_pickle(
    fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\permnos_1Y_p{PF_SIZE}.pickle")

intensities_dfs = {}
for k, v in evs_dfs.items():
    v.index = pd.to_datetime(permnos.index, format="%Y%m%d")
    intensities_dfs[k] = get_ev_intensities(v, evs_dfs["sample"])

In [7]:
import pickle

base_folder_path = r'H:\\all\\RL_Shrinkage_2024'
# IMPORT SHRK DATASETS
pf_size = PF_SIZE  # DONT CHANGE HERE!!
permnos = pd.read_pickle(
    fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\permnos_1Y_p{pf_size}.pickle")
rets_full = pd.read_pickle(
    fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\returns_full_1Y_p{pf_size}.pickle")

fixed_shrk_name = 'cov2Para'
opt_shrk_name = 'cov2Para'
with open(rf"{base_folder_path}\ONE_YR\preprocessing\training_dfs\PF{pf_size}\fixed_shrkges_cov2Para_p{pf_size}.pickle", 'rb') as f:
    fixed_shrk_data = pickle.load(f)
with open(rf"{base_folder_path}\ONE_YR\preprocessing\training_dfs\PF{pf_size}\cov2Para_factor-1.0_p{pf_size}.pickle", 'rb') as f:
    optimal_shrk_data = pickle.load(f)

# get all the validation indices
len_train = 5040
end_date = fixed_shrk_data.shape[0]
# temp here
val_indices_correct = (len_train, end_date)
val_indices_results = [val_indices_correct[0] + 21 * i for i in range((val_indices_correct[-1] - val_indices_correct[0]) // 21)]
val_idxes_shrkges = [0 + 21 * i for i in range((val_indices_correct[-1] - val_indices_correct[0]) // 21)]
reb_date_1 = permnos.index[0]
add_idx = np.where(rets_full.index == reb_date_1)[0][0]

In [8]:
intensity_of_intensitiy_list = np.arange(0.5, 2.01, 0.05).round(2)

In [9]:
def get_new_eigenvalues_v2(qis_eigenvalues, sample_eigenvalues, intensity_of_intensity):
    intensity = qis_eigenvalues.copy()
    if qis_eigenvalues.shape[1] == 500:
        intensity.iloc[: , -251:] = qis_eigenvalues.iloc[: , -251:] / sample_eigenvalues.iloc[: , -251:]
    else:
        intensity = qis_eigenvalues / sample_eigenvalues
    intensity_delta = intensity - 1
    intensity_delta_new = intensity_delta * intensity_of_intensity
    intensity_new = intensity_delta_new + 1
    qis_evs_new = intensity_new * sample_eigenvalues
    if qis_eigenvalues.shape[1] == 500:
        qis_evs_new.iloc[: , 0:250] = intensity_of_intensity * qis_eigenvalues.iloc[: , 0:250]
    return qis_evs_new

In [10]:
qis_evs_new = {}
intensities_new = {}
for intensity_of_intensity in intensity_of_intensitiy_list:
    qis_evs_new[intensity_of_intensity] = get_new_eigenvalues_v2(evs_dfs["qis"], evs_dfs["sample"], intensity_of_intensity)
    intensity = qis_evs_new[intensity_of_intensity].copy()
    if PF_SIZE == 500:
        intensity.iloc[: , -251:] = qis_evs_new[intensity_of_intensity].iloc[: , -251:] / evs_dfs["sample"].iloc[: , -251:]
    else:
        intensity = qis_evs_new[intensity_of_intensity] / evs_dfs["sample"]
    intensities_new[intensity_of_intensity] = intensity.copy()

#### Get the Training data for the new intensity of intensities

In [11]:
#PF_SIZE = 500
try:
    ioi_path = r"H:\all\RL_Shrinkage_2024\ONE_YR\NonLinear_Shrinkage\intensity_of_intensity_data"
    all_res = pd.read_csv(ioi_path + f"\\all_res_p{PF_SIZE}_long_only.csv", index_col=0)
    all_rawres = pd.read_csv(ioi_path + f"\\all_rawres_p{PF_SIZE}_long_only.csv", index_col=0)
except:
    tmp_rawres, tmp_res =  get_rawres_LONG_only(qis_evs_new, modelnames = intensity_of_intensitiy_list)
    all_res = pd.DataFrame(tmp_res.copy())
    all_rawres = pd.DataFrame(tmp_rawres.copy())
    out_path = r"H:\all\RL_Shrinkage_2024\ONE_YR\NonLinear_Shrinkage\intensity_of_intensity_data"
    all_res.to_csv(out_path + f"\\all_res_p{PF_SIZE}_long_only.csv")
    all_rawres.to_csv(out_path + f"\\all_rawres_p{PF_SIZE}_long_only.csv")

Some Other Error..
Some Other Error..
Some Other Error..
Some Other Error..
Some Other Error..
Some Other Error..
Some Other Error..
Some Other Error..
Some Other Error..


In [15]:
weights = hf.calc_global_min_variance_pf_long_only(sigmahat)

NameError: name 'sigmahat' is not defined

In [19]:
if 1==1:
    modelnames = intensity_of_intensitiy_list
    eigenvalue_dict = qis_evs_new
    tmp_res = defaultdict(list)
    tmp_rawres = defaultdict(list)
    for idx in range(0, 1):
        try:
            past_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + add_idx - 21 * 12 * 1: idx + add_idx, :]
            past_ret_mat = past_ret_mat.sub(past_ret_mat.mean())
            past_ret_mat = past_ret_mat.fillna(0)
            fut_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + add_idx: idx + add_idx + 21, :]
        except:
            print("Some Error..")
            
        N, p = past_ret_mat.shape
        sample = pd.DataFrame(np.matmul(past_ret_mat.T.to_numpy(), past_ret_mat.to_numpy())) / (N - 1)
        lambda1, u = np.linalg.eigh(sample)
        lambda1 = lambda1.real.clip(min=0)
        dfu = pd.DataFrame(u,columns=lambda1)
        dfu.sort_index(axis=1,inplace = True)
        temp1 = dfu.to_numpy()
        temp3 = dfu.T.to_numpy().conjugate()

        for cur_modelname in modelnames:
            qis = eigenvalue_dict[cur_modelname].iloc[idx, :]
            temp2 = np.diag(qis)
            sigmahat = pd.DataFrame(np.matmul(np.matmul(temp1, temp2), temp3))
            try:
                weights = hf.calc_global_min_variance_pf_long_only(sigmahat)
            except:
                print("Some Other Error..")

Some Other Error..
Some Other Error..
Some Other Error..
Some Other Error..
Some Other Error..
Some Other Error..
Some Other Error..
Some Other Error..
Some Other Error..


In [20]:
weights = hf.calc_global_min_variance_pf_long_only(sigmahat)

DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
QuadForm(var1537, [[0.00 0.00 ... 0.00 0.00]
 [0.00 0.00 ... 0.00 0.00]
 ...
 [0.00 0.00 ... 0.00 0.00]
 [0.00 0.00 ... 0.00 0.00]])

In [21]:
sigmahat

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.000584,0.000049,0.000083,0.000151,0.000160,0.000114,0.000145,0.000133,0.000085,0.000100,...,0.000152,0.000034,0.000028,0.000035,0.000122,0.000097,0.000056,0.000117,0.000059,0.000073
1,0.000049,0.000486,0.000026,0.000036,0.000050,0.000048,0.000045,0.000051,0.000038,0.000041,...,0.000065,0.000028,0.000025,0.000019,0.000059,0.000057,0.000030,0.000055,0.000025,0.000028
2,0.000083,0.000026,0.000557,0.000153,0.000147,0.000142,0.000149,0.000142,0.000071,0.000082,...,0.000208,0.000045,0.000029,0.000036,0.000111,0.000104,0.000077,0.000126,0.000040,0.000072
3,0.000151,0.000036,0.000153,0.000630,0.000238,0.000193,0.000227,0.000217,0.000114,0.000123,...,0.000287,0.000067,0.000039,0.000033,0.000159,0.000162,0.000107,0.000167,0.000067,0.000117
4,0.000160,0.000050,0.000147,0.000238,0.000640,0.000189,0.000225,0.000242,0.000091,0.000112,...,0.000247,0.000022,0.000017,0.000012,0.000156,0.000132,0.000062,0.000158,0.000046,0.000094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.000097,0.000057,0.000104,0.000162,0.000132,0.000140,0.000149,0.000139,0.000110,0.000117,...,0.000226,0.000068,0.000055,0.000035,0.000164,0.000487,0.000089,0.000167,0.000084,0.000108
496,0.000056,0.000030,0.000077,0.000107,0.000062,0.000099,0.000094,0.000099,0.000059,0.000070,...,0.000178,0.000043,0.000047,0.000039,0.000096,0.000089,0.000377,0.000102,0.000057,0.000052
497,0.000117,0.000055,0.000126,0.000167,0.000158,0.000164,0.000171,0.000165,0.000109,0.000123,...,0.000251,0.000091,0.000030,0.000055,0.000164,0.000167,0.000102,0.000384,0.000057,0.000125
498,0.000059,0.000025,0.000040,0.000067,0.000046,0.000086,0.000059,0.000057,0.000063,0.000059,...,0.000112,0.000026,0.000051,0.000037,0.000086,0.000084,0.000057,0.000057,0.000279,0.000056


In [12]:
import cvxpy as cp

In [48]:
def MinVar_longshort(covmat):
    w = cp.Variable(covmat.shape[0])  # because wgbi weights has 24 non na entries
    risk = cp.quad_form(w, covmat)
    objective = cp.Minimize(risk)
    constraints = [cp.sum(w) == 1]
    prob = cp.Problem(objective, constraints)
    prob.solve()
    # result is stored in the cp.Variable (w) after solving
    return w.value

In [46]:
def MinVar_long_only(covmat):
    w = cp.Variable(covmat.shape[0])  # because wgbi weights has 24 non na entries
    risk = cp.quad_form(w, covmat)
    objective = cp.Minimize(risk)
    constraints = [cp.sum(w) == 1, w >= 0]
    prob = cp.Problem(objective, constraints)
    prob.solve()
    # result is stored in the cp.Variable (w) after solving
    return w.value

In [86]:
covmat = sigmahat
w = cp.Variable(covmat.shape[0])  
risk = cp.quad_form(w, covmat.values)
objective = cp.Minimize(risk)
constraints = [cp.sum(w) == 1, w >= 0]
prob = cp.Problem(objective)
prob.solve()

DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
QuadForm(var1807, [[0.00 0.00 ... 0.00 0.00]
 [0.00 0.00 ... 0.00 0.00]
 ...
 [0.00 0.00 ... 0.00 0.00]
 [0.00 0.00 ... 0.00 0.00]])

In [81]:
covmat

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.000584,0.000049,0.000083,0.000151,0.000160,0.000114,0.000145,0.000133,0.000085,0.000100,...,0.000152,0.000034,0.000028,0.000035,0.000122,0.000097,0.000056,0.000117,0.000059,0.000073
1,0.000049,0.000486,0.000026,0.000036,0.000050,0.000048,0.000045,0.000051,0.000038,0.000041,...,0.000065,0.000028,0.000025,0.000019,0.000059,0.000057,0.000030,0.000055,0.000025,0.000028
2,0.000083,0.000026,0.000557,0.000153,0.000147,0.000142,0.000149,0.000142,0.000071,0.000082,...,0.000208,0.000045,0.000029,0.000036,0.000111,0.000104,0.000077,0.000126,0.000040,0.000072
3,0.000151,0.000036,0.000153,0.000630,0.000238,0.000193,0.000227,0.000217,0.000114,0.000123,...,0.000287,0.000067,0.000039,0.000033,0.000159,0.000162,0.000107,0.000167,0.000067,0.000117
4,0.000160,0.000050,0.000147,0.000238,0.000640,0.000189,0.000225,0.000242,0.000091,0.000112,...,0.000247,0.000022,0.000017,0.000012,0.000156,0.000132,0.000062,0.000158,0.000046,0.000094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.000097,0.000057,0.000104,0.000162,0.000132,0.000140,0.000149,0.000139,0.000110,0.000117,...,0.000226,0.000068,0.000055,0.000035,0.000164,0.000487,0.000089,0.000167,0.000084,0.000108
496,0.000056,0.000030,0.000077,0.000107,0.000062,0.000099,0.000094,0.000099,0.000059,0.000070,...,0.000178,0.000043,0.000047,0.000039,0.000096,0.000089,0.000377,0.000102,0.000057,0.000052
497,0.000117,0.000055,0.000126,0.000167,0.000158,0.000164,0.000171,0.000165,0.000109,0.000123,...,0.000251,0.000091,0.000030,0.000055,0.000164,0.000167,0.000102,0.000384,0.000057,0.000125
498,0.000059,0.000025,0.000040,0.000067,0.000046,0.000086,0.000059,0.000057,0.000063,0.000059,...,0.000112,0.000026,0.000051,0.000037,0.000086,0.000084,0.000057,0.000057,0.000279,0.000056


In [98]:
evs_dfs["qis"].iloc[-1, 255]

9.289966803749822e-05

In [99]:
evs_dfs["sample"].iloc[-1, 255]

2.0858393415148968e-05

In [92]:
evs_dfs["qis"].iloc[-1, 255] / evs_dfs["sample"].iloc[-1, 255]

4.453826629333175

In [93]:
evs_dfs["qis"].iloc[-1, 255] / evs_dfs["sample"].iloc[-1, 255] - 1

3.453826629333175

In [94]:
(evs_dfs["qis"].iloc[-1, 255] / evs_dfs["sample"].iloc[-1, 255] - 1)*2

6.90765325866635

In [95]:
(evs_dfs["qis"].iloc[-1, 255] / evs_dfs["sample"].iloc[-1, 255] - 1)*2 + 1

7.90765325866635

In [97]:
((evs_dfs["qis"].iloc[-1, 255] / evs_dfs["sample"].iloc[-1, 255] - 1)*2 + 1) * evs_dfs["sample"].iloc[-1, 255]

0.00016494094265984748